In [ ]:
!pip install beautifulsoup4
!pip install pandas
!pip install selenium
!pip install webdriver-manager

In [3]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [4]:
os.makedirs("player_stats",exist_ok=True)

In [ ]:
service = Service(ChromeDriverManager().install())

In [ ]:
def extract_player_stats(year):
    next_year = year + 1
    player_stats_url = f"https://fbref.com/en/comps/9/{year}-{next_year}/stats/{year}-{next_year}-Premier-League-Stats"
    table_id = "stats_standard"

    driver = webdriver.Chrome(service=service)
    driver.get(player_stats_url)

    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, table_id))
        )
        page_source = driver.page_source
        with open(f"player_stats/player_stats_{year}.html", "w+", encoding="utf-8") as f:
            f.write(page_source)

        table = driver.find_element(By.ID, table_id)
        html_content = table.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content, "html.parser")


        for interval_header in soup.find_all('thead')[1:]:
            interval_header.decompose()


        header_rows = soup.find_all('thead')[0].find_all('tr')
        headers = []
        for header_row in header_rows:
            header_cols = header_row.find_all(['th', 'td'])
            headers.append([header_col.text.strip() for header_col in header_cols])
        headers = headers[-1]


        rows = []
        for row in soup.find_all('tbody')[0].find_all('tr'):
            cells = row.find_all(['th', 'td'])
            if len(cells) > 0:
                row_data = [cell.text.strip() for cell in cells]
                rows.append(row_data)

        return headers, rows

    except Exception as e:
        print(f"Table did not load for {year}-{next_year}: {e}")
        return [], []

    finally:
        driver.quit()

for year in range(1992, 2024):
    headers, year_data = extract_player_stats(year)


    if year_data:
        df = pd.DataFrame(year_data, columns=headers)
        df.to_csv(f'player_stats/player_stats_{year}_{year+1}.csv', index=False)
        print(f"Data scraping completed and saved to player_stats_{year}_{year+1}.csv")
    else:
        print(f"No data found for {year}-{year+1}.")


next step is to create an additional column of years and seasons for each season.

In [4]:
player_stats_path = "player_stats"
player_df_list = []

for year in range(1992, 2024):
    csv_file = os.path.join(player_stats_path, f"player_stats_{year}_{year+1}.csv")
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)

        df['year'] = year

        df['season'] = f"{year}/{year+1}"

        player_df_list.append(df)
    else:
        print(f"File {csv_file} does not exist")

player_merged_df = pd.concat(player_df_list, ignore_index=True)

output_dir = "player_stats"
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, "merged_player_stats.csv")
player_merged_df.to_csv(output_file, index=False)

print("All csv files have been merged and saved to player_merged_df")

All csv files have been merged and saved to player_merged_df


merge the poy data to the merged player csv

In [15]:
merged_player_stats_path = "player_stats/merged_player_stats.csv"
poy_path = "poy/Player_of_year.csv"

In [16]:
merged_player_stats_df = pd.read_csv(merged_player_stats_path, low_memory=False)
poy_df = pd.read_csv(poy_path)

In [17]:
print("Columns of merged_player_stats_df:", merged_player_stats_df.columns)
print("Columns of poy_df:", poy_df.columns)


Columns of merged_player_stats_df: Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts',
       'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY',
       'CrdR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'Matches',
       'year', 'season', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP',
       'PrgR', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1'],
      dtype='object')
Columns of poy_df: Index(['year', 'season', 'Player', 'Squad'], dtype='object')


checking for inconsistent season entry

In [18]:
poy_df["season"]

0     1992/1993
1     1993/1994
2     1994/1995
3     1995/1996
4     1996/1997
5     1997/1998
6     1998/1999
7     1999/2000
8     2000/2001
9     2001/2002
10    2002/2003
11    2003/2004
12    2004/2005
13    2005/2006
14    2006/2007
15    2007/2008
16    2008/2009
17    2009/2010
18    2010/2011
19    2011/2012
20    2012/2013
21    2013/2014
22    2014/2015
23    2015/2016
24    2016/2017
25    2017/2018
26    2018/2019
27    2019/2020
28    2020/2021
29    2021/2022
30    2022/2023
31    2023/2024
Name: season, dtype: object

In [19]:
final_merged_df = pd.merge(merged_player_stats_df, poy_df, how = 'left', on=['Player', 'season'])
final_merged_df.to_csv("final_merged_stats.csv", index=False)
print("Merged data saved")

Merged data saved
